<h2>Liconic STX Series</h2>

The Liconic STX line of automated incubators come in a variety of sizes including STX 1000, STX 500, STX 280, STX 220, STX 110, STX 44. Which corresponds to the number of plates each size can store using the standard 22 plate capacity cassettes/cartridges (plate height 17mm, 505mm total height.) There are other cassette size for plates height ranging from 5 to 104mm in height (higher plates = less number of plates storage capacity.)

The Liconic STX line comes in a variety of climate control options including Ultra High Temp. (HTT), Incubator (IC), Dry Storage (DC2), Humid Cooler (HC), Humid Wide Range (HR), Dry Wide Range (DR2), Humidity Controlled (AR), Deep Freezer (DF) and Ultra Deep Freezer (UDF). Each have different ranges of temperatures and humidity control ability.

Other accessories that can be included with the STX and can be utilized with this driver include N2 gassing, CO2 gassing, a Turn Station (rotation of plates 90 degrees on the transfer station), internal barcode scanners, a swap station (two transfer plate positions that can be rotated 180 degrees) and internal shaking. 

This tutorial shows how to
 - Connect the Liconic incubator
 - Configure racks
 - Move plates in and out
 - Set and monitor temperature and humidity values

In [ ]:

from pylabrobot.resources.coordinate import Coordinate
from pylabrobot.storage import LiconicBackend
from pylabrobot.storage.incubator import Incubator
from pylabrobot.storage.liconic.racks import liconic_rack_17mm_22, liconic_rack_44mm_10

backend = LiconicBackend(port="COM3", model="STX220_HC", barcode_installed=True, barcode_port="COM4")

rack = [
        liconic_rack_44mm_10("cassette_0"),
        liconic_rack_44mm_10("cassette_1"),
        liconic_rack_44mm_10("cassette_2"),
        liconic_rack_17mm_22("cassette_3"),
        liconic_rack_17mm_22("cassette_4"),
        liconic_rack_17mm_22("cassette_5"),
        liconic_rack_17mm_22("cassette_6"),
        liconic_rack_17mm_22("cassette_7"),
        liconic_rack_17mm_22("cassette_8"),
        liconic_rack_17mm_22("cassette_9")
        ]

incubator = Incubator(
  backend=backend,
  name="My Incubator",
  size_x=100,
  size_y=100,
  size_z=100,
  racks=rack,
  loading_tray_location=Coordinate(x=0, y=0, z=0),
)

await incubator.setup()

Setup

To setup the incubator and start sending commands first the backed needs to be declared. For the Liconic the LiconcBackend class is used with the COM port used for connection (in this case COM3) and the model needs to specified (in this case the STX 220 Humid Cooler, STX220_HC). If an internal barcode is installed the barcode_installed parameter is set to True and its COM port is also specified. These two parameters are optional so can be omitted for Liconics without an internal barcode scanner. 

Given a STX 220 (220 plate position / 22 plates per rack = 10 racks) can hold 10 racks the list of racks is built and includes mixing and matching different plate height racks. The differences in racks are handled prior to plate retrieval and storage. 

Once the these are built the base Incubator class is created and the connection to the incubator is initialized using:

In [ ]:
await incubator.setup()

To store a plate first a plate resource is initialized and then assigned to the loading tray. The method take_in_plate is then called on the incubator object.



In [ ]:
from pylabrobot.resources import Azenta4titudeFrameStar_96_wellplate_200ul_Vb

new_plate = Azenta4titudeFrameStar_96_wellplate_200ul_Vb(name="TEST")
incubator.loading_tray.assign_child_resource(new_plate)
await incubator.take_in_plate("smallest")  # choose the smallest free site

# other options:
# await incubator.take_in_plate("random")  # random free site
# await incubator.take_in_plate(rack[3])  # store at rack position 3

Barcode can returned 

In [ ]:
await incubator.move_position_to_position(plate_name="TEST",dest_site=position,read_barcode=True)

To retrieve a plate the plate name can used

In [ ]:
await incubator.fetch_plate_to_loading_tray(plate="TEST")
retrieved = incubator.loading_tray.resource

You can also print a barcode from this call (if barcode is installed per the backend insatiation). Returning of the barcode as a return object still needs to be implemented. 

In [ ]:
await incubator.fetch_plate_to_loading_tray(plate_name="TEST",read_barcode=True)
# will print the barcode to the terminal

The humdity, temperature, N2 gas and CO2 gas levels can all be controlled and queried. For temperature for example:

- To get the current temperature

In [ ]:
temperature = await incubator.get_temperature() # returns temperature as float in Celsius to the 10th place
print(str(temperature))

- To set the temperature of the Liconic

In [ ]:
await incubator.set_temperature(8.0) # set the temperature to 8 degrees Celsius

- You can also retrieve the set value (the value sent for set_temperature) using:

In [ ]:
set_temperature = await incubator.get_set_temperature() # will return a float for the set temperature in degrees Celsius

This pattern is the same for CO2, N2 and Humidity control of the Liconic. 